# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost/companies") 
db = client.get_database()

In [2]:
list(db.list_collections())

[{'name': 'companies',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': UUID('6b291988-e92e-4520-b53e-cc7e46824744')},
  'idIndex': {'v': 2,
   'key': {'_id': 1},
   'name': '_id_',
   'ns': 'companies.companies'}}]

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [24]:
query = {"name":{"$eq":"Babelgum"}}
list(db["companies"].find(query,{"_id":0,"name":1}))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [38]:
query2 = {"number_of_employees":{"$gte":5000}}
list(db["companies"].find(query2).sort("number_of_employees",1).limit(20))

[{'_id': ObjectId('52cdef7d4bab8bd67529891b'),
  'name': 'Spotify',
  'permalink': 'spotify',
  'crunchbase_url': 'http://www.crunchbase.com/company/spotify',
  'homepage_url': 'http://www.spotify.com',
  'blog_url': 'http://www.spotify.com/blog/',
  'blog_feed_url': 'http://www.spotify.com/blog/feed',
  'twitter_username': 'spotify',
  'category_code': 'music',
  'number_of_employees': 5000,
  'founded_year': 2006,
  'founded_month': None,
  'founded_day': None,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': 'music-streaming, streaming, music, cloud',
  'alias_list': '',
  'email_address': 'press@spotify.com',
  'phone_number': '0709821244',
  'description': 'Music streaming over the internet',
  'created_at': 'Mon Apr 28 19:10:19 UTC 2008',
  'updated_at': 'Thu Dec 12 00:45:40 UTC 2013',
  'overview': '<p>Spotify has created a lightweight software application that allows instant listening to specific tracks or 

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [47]:
query3 = {"$and":[{"founded_year":{"$gte":2000}},{"founded_year":{"$lte":2005}}]}
list(db["companies"].find(query3,{"_id":0,"name":1,"founded_year":1}).limit(5))

[{'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Gizmoz', 'founded_year': 2003}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [74]:
query4 = {"$and":[{"ipo.valuation_amount":{"$gte":100000000}}, {"founded_year":{"$gte":2000}}]}
list(db["companies"].find(query4,{"_id":0,"name":1,"ipo.valuation_amount":1}).sort("nipo.valuation_amount",-1).limit(5))

[{'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Yelp', 'ipo': {'valuation_amount': 1300000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000}},
 {'name': 'Brightcove', 'ipo': {'valuation_amount': 290000000}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [75]:
query5 = {"$and":[{"number_of_employees":{"$lte":1000}}, {"founded_year":{"$lte":2005}}]}
list(db["companies"].find(query5,{"_id":0,"name":1,"number_of_employees":1, "founded_year":1}).sort("number_of_employees",-1).limit(10))

[{'name': 'Akamai Technologies',
  'number_of_employees': 1000,
  'founded_year': 1998},
 {'name': 'Omniture', 'number_of_employees': 1000, 'founded_year': 1996},
 {'name': 'Yodle', 'number_of_employees': 1000, 'founded_year': 2005},
 {'name': 'Sonus Networks', 'number_of_employees': 1000, 'founded_year': 1997},
 {'name': 'Telenav', 'number_of_employees': 1000, 'founded_year': 1999},
 {'name': 'Antal International',
  'number_of_employees': 1000,
  'founded_year': 1993},
 {'name': 'SK Net Service Company Ltd',
  'number_of_employees': 1000,
  'founded_year': 2001},
 {'name': 'Gumtree', 'number_of_employees': 1000, 'founded_year': 2005},
 {'name': 'FANUC Robotics America',
  'number_of_employees': 1000,
  'founded_year': 1982},
 {'name': "OC'LIANE", 'number_of_employees': 1000, 'founded_year': 2003}]

### 6. All the companies that don't include the `partners` field.

In [76]:
query6 = {"partners": {"$exists": False }}
list(db["companies"].find(query6))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [77]:
query7 = {"category_code": { "$type": "null" }}
list(db["companies"].find(query7,{"_id":0,"name":1,"category_code":1}).limit(5))

[{'name': 'Collective', 'category_code': None},
 {'name': 'Snimmer', 'category_code': None},
 {'name': 'KoolIM', 'category_code': None},
 {'name': 'VidKing', 'category_code': None},
 {'name': 'Level9 Media', 'category_code': None}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [78]:
query8 = {"$and":[{"number_of_employees":{"$gte":100}}, {"number_of_employees":{"$lte":1000}}]}
list(db["companies"].find(query8,{"_id":0,"name":1,"number_of_employees":1}).limit(5))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110}]

### 9. Order all the companies by their IPO price in a descending order.

In [80]:
query9= {"ipo.valuation_amount":{"$gte":0}}
list(db["companies"].find(query9,{"_id":0,"name":1,"ipo.valuation_amount":1}).sort("ipo.valuation_amount",-1).limit(5))

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [106]:
query10 = {"number_of_employees":{"$gte":0}}
list(db["companies"].find(query10,{"_id":0,"name":1,"number_of_employees":1}).sort("number_of_employees",-1).limit(10))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [108]:
query11 = {"founded_month":{"$gte":7}}
lista11 = list(db["companies"].find(query11,{"_id":0,"name":1}).limit(1000))

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [4]:
query12 = {"$and":[{"founded_year":{"$lte":2000}},{"ipo.valuation_amount":{"$gte":10000}}]}
list(db["companies"].find(query12,{"_id":0,"name":1,"ipo.valuation_amount":1}).sort("number_of_employees",-1).limit(10))

[{'name': 'Baidu', 'ipo': {'valuation_amount': 4000000000}},
 {'name': 'Salesforce', 'ipo': {'valuation_amount': 110000000}},
 {'name': 'DeNA', 'ipo': {'valuation_amount': 117900}},
 {'name': 'DeNA', 'ipo': {'valuation_amount': 117900}},
 {'name': 'Telenav', 'ipo': {'valuation_amount': 337000000}},
 {'name': 'Cornerstone OnDemand', 'ipo': {'valuation_amount': 900000000}},
 {'name': 'Shutterfly', 'ipo': {'valuation_amount': 350000000}},
 {'name': 'QlikTech', 'ipo': {'valuation_amount': 1000000000}},
 {'name': 'OpenTable', 'ipo': {'valuation_amount': 1050000000}},
 {'name': 'QuinStreet', 'ipo': {'valuation_amount': 140000000}}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [6]:
query13 = {"acquisition.acquired_year":{"$gte":2010}}
list(db["companies"].find(query13,{"_id":0,"name":1,"acquisition.acquired_year":1}).limit(10))

[{'name': 'Wetpaint', 'acquisition': {'acquired_year': 2013}},
 {'name': 'Digg', 'acquisition': {'acquired_year': 2012}},
 {'name': 'Geni', 'acquisition': {'acquired_year': 2012}},
 {'name': 'Kyte', 'acquisition': {'acquired_year': 2011}},
 {'name': 'Veoh', 'acquisition': {'acquired_year': 2010}},
 {'name': 'Jingle Networks', 'acquisition': {'acquired_year': 2011}},
 {'name': 'blogTV', 'acquisition': {'acquired_year': 2013}},
 {'name': 'delicious', 'acquisition': {'acquired_year': 2011}},
 {'name': 'Revision3', 'acquisition': {'acquired_year': 2012}},
 {'name': 'Jobster', 'acquisition': {'acquired_year': 2010}}]

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [7]:
query14 = {"founded_year":{"$gte":1}}
list(db["companies"].find(query14,{"_id":0,"name":1,"founded_year":1}).sort("founded_year",-1).limit(10))

[{'name': 'Fixya', 'founded_year': 2013},
 {'name': 'Wamba', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Gimigo', 'founded_year': 2013},
 {'name': 'SEOGroup', 'founded_year': 2013},
 {'name': 'Clowdy', 'founded_year': 2013},
 {'name': 'WhosCall', 'founded_year': 2013},
 {'name': 'Pikk', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [9]:
query15 = {"founded_day":{"$lt":8}}
list(db["companies"].find(query15,{"_id":0,"name":1,"founded_year":1, "acquisition_price":1}).sort("acquisition_price",-1).limit(10))

[{'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'Postini', 'founded_year': 1999},
 {'name': 'Geni', 'founded_year': 2006},
 {'name': 'Fox Interactive Media', 'founded_year': 1979},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'Joost', 'founded_year': 2006},
 {'name': 'Babelgum', 'founded_year': 2007},
 {'name': 'Facebook', 'founded_year': 2004}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [10]:
query16 = {"$and":[{"category_code":"web"},{"number_of_employees":{"$gte":4000}}]}
list(db["companies"].find(query15,{"_id":0,"name":1, "number_of_employees":1}).sort("number_of_employees",-1).limit(10))

[{'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'MetaCarta', 'number_of_employees': 99999},
 {'name': 'Microsoft', 'number_of_employees': 90000},
 {'name': 'Apple', 'number_of_employees': 80000},
 {'name': 'Google', 'number_of_employees': 28000},
 {'name': 'eBay', 'number_of_employees': 15000},
 {'name': 'Harris', 'number_of_employees': 15000},
 {'name': 'Yahoo!', 'number_of_employees': 13600}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [12]:
query17 = {"$and":[{"acquisition.price_amount":{"$gte":10000}}, {"acquisition.price_currency_code":{"$eq":'EUR'}}]}
list(db["companies"].find(query17,{"_id":0,"name":1, "acquisition.price_currency_code":1}).limit(5))

[{'name': 'ZYB', 'acquisition': {'price_currency_code': 'EUR'}},
 {'name': 'Apertio', 'acquisition': {'price_currency_code': 'EUR'}},
 {'name': 'Greenfield Online', 'acquisition': {'price_currency_code': 'EUR'}},
 {'name': 'Mobile Trend', 'acquisition': {'price_currency_code': 'EUR'}},
 {'name': 'Citizenside', 'acquisition': {'price_currency_code': 'EUR'}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [13]:
query18 = {"acquisition.acquired_month":{"$lte":3}}
list(db["companies"].find(query18,{"_id":0,"name":1, "acquisition.acquired_month":1}).limit(10))

[{'name': 'Kyte', 'acquisition': {'acquired_month': 1}},
 {'name': 'NetRatings', 'acquisition': {'acquired_month': 2}},
 {'name': 'blogTV', 'acquisition': {'acquired_month': 3}},
 {'name': 'Livestream', 'acquisition': {'acquired_month': 1}},
 {'name': 'iContact', 'acquisition': {'acquired_month': 2}},
 {'name': 'Coghead', 'acquisition': {'acquired_month': 2}},
 {'name': 'Dailymotion', 'acquisition': {'acquired_month': 2}},
 {'name': 'Netvibes', 'acquisition': {'acquired_month': 2}},
 {'name': 'Flickr', 'acquisition': {'acquired_month': 3}},
 {'name': 'BabyCenter', 'acquisition': {'acquired_month': 3}}]

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code